## **Распределительная задача**

Завод может производить Y = 25 видов продукции. На продукцию есть покупатель, которому требуется определенное количество каждого вида:

Таблица M (кг): M = [52 37 65 64 66 66 62 49 33 54 49 67 60 68 69 35 45 53 62 40 32 54 64 60 47]

Для каждого вида продукции он либо покупает требуемое количество, либо не покупает совсем, если мы произвели недостаточно. При этом известна цена контракта на каждый вид продукции:

Таблица C (рубли): C = [656 672 675 665 512 620 451 538 523 670 591 421 574 531 608 454 471 414 605 543 612 514 638 444 437]

Завод одновременно может выпускать только один вид продукции со скоростью:

Таблица P (кг в сутки): P = [19 15 27 19 29 16 22 18 16 27 21 21 24 25 28 21 24 21 15 15 17 29 27 24 21]

Требуется написать математическую модель и найти оптимальный план производства на S = 20 суток с помощью Gurobi, если известно, что переход от одной продукции к другой происходит мгновенно, а его стоимость R = 49 рублей.


#### Запишем математическую модель:

Введём переменные:

$$x_{i} = \begin{cases}
   1,&\text{если заключаем контракт с } i &\text{заводом,}\\
   0,&\text{в противном случае.}
\end{cases}$$

Целевая функция:

$$\sum\limits_{i = 0}^{Y} (c_i - R)x_i + R\to \max_{x}$$

Ограничения:

$$x_i \in \mathbb B, \forall i \in I $$

$$\sum\limits_{i = 0}^{Y} x_i \lceil m_i/p_i \rceil \leqslant S $$